# PnL Reports for Interactive Brokers Flex Queries
Run flex query in [IB account management](https://gdcdyn.interactivebrokers.com/sso/Login). Modify test_transfer_query run dates so that last day of report is last day of the month to ensure all fields are present
    
- daily_nav : YTD Daily total return calculation for whole portolio    
- ytd_comprehensive: Need to first update date of flex report before running it at month end Detailed performance breakdown (uses [FlexStatement class](https://github.com/codetradr/ib-flex-analyzer) from Wes Mckinney)



In [1]:
import pandas as pd
from pathlib import Path
import lxml.objectify as lxml_objectify
import numpy as np
import pyfolio as pf
import sys
opt_path = '/Users/ebellord/PycharmProjects/opt_trade'
if  opt_path not in sys.path:
    sys.path.append(opt_path)
from account_management import FlexStatement       
file_name = 'daily_nav.xml'

tree = lxml_objectify.parse(open(Path.home() / 'Downloads' / file_name, 'rb'))
root = tree.getroot()
stmt = root.FlexStatements.FlexStatement

In [12]:
# Withdrawals and deposits
node = stmt.CashTransactions
cash_trans = pd.DataFrame([dict(zip(c.keys(), c.values())) for
                           c in node.getchildren()])
cash_trans.amount = cash_trans.amount.astype(np.float64)
cash_trans.dateTime = pd.to_datetime(cash_trans.dateTime)
cash_trans = cash_trans.groupby('dateTime').sum().squeeze()

# Transfers
# node = stmt.Transfers
# transfers = pd.DataFrame([dict(zip(c.keys(), c.values())) for
#                          c in node.getchildren()])
# transfers.reportDate = pd.to_datetime(transfers.reportDate)
# transfers = transfers[['assetCategory', 'reportDate', 'positionAmountInBase',
#                        'cashTransfer', 'direction']]
# transfers['total'] = transfers.positionAmountInBase.astype(np.float) \
#                      + transfers.cashTransfer.astype(np.float)
# transfers = transfers.groupby('reportDate').sum()
#
# # Transfers total
# all_transfers = transfers.add(cash_trans, fill_value=0).fillna(0).sum(axis=1)

all_transfers = cash_trans

# Change in NAV
node = stmt.EquitySummaryInBase
daily_nav = pd.DataFrame([dict(zip(c.keys(), c.values())) for
                         c in node.getchildren()])
daily_nav.reportDate = pd.to_datetime(daily_nav.reportDate)
daily_nav = daily_nav.set_index(daily_nav.reportDate)
daily_nav['total'] = daily_nav['total'].astype(np.float)
all_transfers = all_transfers.reindex(daily_nav.index, fill_value=0)

daily_nav['AdjTotal'] = daily_nav['total'].shift(1)
daily_nav['AdjTotal'] = daily_nav['AdjTotal'].add(all_transfers)
daily_nav['TotalReturn'] = daily_nav['total'] / daily_nav['AdjTotal'] - 1
pf.timeseries.aggregate_returns(daily_nav['TotalReturn'].dropna(), 'monthly')
# (daily_nav['AdjTotal'][-1]-daily_nav['AdjTotal'][1]- all_transfers.sum())/3


2019  1   -0.003769
      2    0.019343
      3   -0.020552
Name: TotalReturn, dtype: float64

In [32]:
performance_report = FlexStatement(Path.home() / 'Downloads' / 'ytd_comprehensive.xml')
performance_report.perf.style

,accountId,acctAlias,assetCategory,conid,cusip,description,expiry,isin,issuer,listingExchange,model,mtmMTD,mtmYTD,multiplier,principalAdjustFactor,putCall,realLTMTD,realLTYTD,realSTMTD,realSTYTD,realizedPnlMTD,realizedPnlYTD,securityID,securityIDType,strike,symbol,underlyingConid,underlyingListingExchange,underlyingSecurityID,underlyingSymbol
0,U2463043,,STK,169041186,,CALITHERA BIOSCIENCES INC,,,,NASDAQ,,9520,21840,1,,,0,0,0,0,0,0,,,,CALA,,,,
1,U2463043,,STK,222038336,,PROTEOSTASIS THERAPEUTICS IN,,,,NASDAQ,,-20020,-13860,1,,,0,0,0,0,0,0,,,,PTI,,,,
2,U2463043,,STK,229726228,,VANECK VECTORS RUSSIA ETF,,,,ARCA,,121.68,1250.6,1,,,0,0,0,0,0,0,,,,RSX,,,,
3,U2463043,,OPT,333928910,,SPX 18JAN19 2300.0 P,20190118,,,,,0,24744.2,100,,P,0,0,0,54842.9,0,54842.9196,,,2300,SPXW 190118P02300000,416904,,,SPX
4,U2463043,,OPT,290075820,,SPX 18JAN19 2500.0 C,20190118,,,,,0,-61574.8,100,,C,0,0,0,-61909.3,0,-61909.3458,,,2500,SPX 190118C02500000,416904,,,SPX
5,U2463043,,OPT,353273495,,SPX 20MAR19 2820.0 C,20190320,,,,,3578.36,3578.36,100,,C,0,0,3578.36,3578.36,3578.356,3578.356,,,2820,SPXW 190320C02820000,416904,,,SPX
6,U2463043,,OPT,335686882,,SPX 29MAR19 2840.0 C,20190329,,,,,7794.68,7794.68,100,,C,0,0,0,0,0,0,,,2840,SPXW 190329C02840000,416904,,,SPX
7,U2463043,,OPT,347347837,,SPX 18APR19 2410.0 P,20190418,,,,,0,32728.1,100,,P,0,0,0,32728.1,0,32728.1097,,,2410,SPX 190418P02410000,416904,,,SPX
8,U2463043,,OPT,347347952,,SPX 18APR19 2555.0 P,20190418,,,,,7201.57,21810.5,100,,P,0,0,21810.5,21810.5,21810.516,21810.516,,,2555,SPX 190418P02555000,416904,,,SPX
9,U2463043,,OPT,340221653,,SPX 30APR19 2680.0 P,20190430,,,,,4012.81,4012.81,100,,P,0,0,0,0,0,0,,,2680,SPXW 190430P02680000,416904,,,SPX


In [15]:
performance_report.cash_transactions.style


,accountId,acctAlias,amount,assetCategory,clientReference,code,conid,currency,cusip,dateTime,description,expiry,fxRateToBase,isin,issuer,listingExchange,model,multiplier,principalAdjustFactor,putCall,reportDate,securityID,securityIDType,strike,symbol,tradeID,transactionID,type,underlyingConid,underlyingListingExchange,underlyingSecurityID,underlyingSymbol
0,U2463043,,0.01,FUND,,,141157215,USD,,2018-12-31 00:00:00,693390445(693390445) CASH DIVIDEND 0E-8 USD PER SHARE (Ordinary Dividend),,1,,,,,1,,,20190108,,,,PTTAX,,10080200179,Dividends,,,,
1,U2463043,,184.88,FUND,,,48013360,USD,,2019-01-02 00:00:00,92204X108(US92204X1081) CASH DIVIDEND 0E-8 USD PER SHARE (Ordinary Dividend),,1,,,,,1,,,20190102,,,,VMATX,,10049470060,Dividends,,,,
2,U2463043,,2014.71,FUND,,,48013054,USD,,2019-01-02 00:00:00,VMMXX(US9229062016) CASH DIVIDEND 0E-8 USD PER SHARE (Ordinary Dividend),,1,,,,,1,,,20190103,,,,VMMXX,,10059889846,Dividends,,,,
3,U2463043,,0.01,FUND,,,141157215,USD,,2019-01-31 00:00:00,693390445(693390445) CASH DIVIDEND 0.02633937 USD PER SHARE (Ordinary Dividend),,1,,,,,1,,,20190204,,,,PTTAX,,10198724458,Dividends,,,,
4,U2463043,,185.74,FUND,,,48013360,USD,,2019-02-01 00:00:00,92204X108(US92204X1081) CASH DIVIDEND 0.02812695 USD PER SHARE (Ordinary Dividend),,1,,,,,1,,,20190201,,,,VMATX,,10192115761,Dividends,,,,
5,U2463043,,2096.09,FUND,,,48013054,USD,,2019-02-01 00:00:00,VMMXX(US9229062016) CASH DIVIDEND 0.00219243 USD PER SHARE (Ordinary Dividend),,1,,,,,1,,,20190201,,,,VMMXX,,10192148026,Dividends,,,,
6,U2463043,,0.01,FUND,,,141157215,USD,,2019-02-28 00:00:00,693390445(693390445) CASH DIVIDEND 0.02437418 USD PER SHARE (Ordinary Dividend),,1,,,,,1,,,20190304,,,,PTTAX,,10331846381,Dividends,,,,
7,U2463043,,167.69,FUND,,,48013360,USD,,2019-03-01 00:00:00,92204X108(US92204X1081) CASH DIVIDEND 0.02532673 USD PER SHARE (Ordinary Dividend),,1,,,,,1,,,20190301,,,,VMATX,,10324446249,Dividends,,,,
8,U2463043,,1899.95,FUND,,,48013054,USD,,2019-03-01 00:00:00,VMMXX(US9229062016) CASH DIVIDEND 0.00187954 USD PER SHARE (Ordinary Dividend),,1,,,,,1,,,20190301,,,,VMMXX,,10324450133,Dividends,,,,
9,U2463043,,-1,,,,,EUR,,2019-03-18 00:00:00,WITHDRAWAL FEE,,1.1341,,,,,0,,,20190318,,,,,,10404145154,Other Fees,,,,


In [16]:
performance_report.mtm_ytd

,Stocks,Options,Dividends,Total
CALA,21840.0,0.0000,0.00,21840.0000
PTI,-13860.0,0.0000,0.00,-13860.0000
PTTAX,0.0,0.0000,0.03,0.0300
RSX,1250.6,0.0000,0.00,1250.6000
SPX,0.0,56124.9169,0.00,56124.9169
VMATX,0.0,0.0000,538.31,538.3100
VMMXX,0.0,0.0000,6010.75,6010.7500


In [17]:
performance_report.cash_by_type

type
Broker Interest Received      8570.91
Deposits/Withdrawals       -222500.00
Dividends                     6549.09
Other Fees                     -28.75
Name: amount, dtype: float64

In [19]:
performance_report.realized

,Stocks,Options,Dividends,Total
CALA,0.0,0.0000,0.00,0.0000
PTI,0.0,0.0000,0.00,0.0000
PTTAX,0.0,0.0000,0.03,0.0300
RSX,0.0,0.0000,0.00,0.0000
SPX,0.0,66381.0643,0.00,66381.0643
VMATX,0.0,0.0000,538.31,538.3100
VMMXX,0.0,0.0000,6010.75,6010.7500


In [29]:
performance_report.option_perf['mtmYTD'].sum()

56124.91689999999